<a href="https://colab.research.google.com/github/Cogitarian/MultCorp/blob/main/EAF2CSV2MP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Po wykonaniu anotacji przydatne jest zestawić fragmenty filmów, które mają takie same etykiety lub według innych kryteriów. Wówczas pomocna będzie operacja, która podzieli film na fragmenty na podstawie pliku .eaf. Operacja składa się z dwóch skryptów. Pierwszy przetwarza dane z pliku .eaf, z którego pobierze nazwę próbki oraz początek i koniec próbki, i zachowuje je w pliku .csv.:

In [ ]:
#!/bin/bash
eaf_file="$1"
if [[ ! -f "$eaf_file" ]]; then
    echo "EAF file not found!"
    exit 1
fi
output_csv="${eaf_file%.eaf}.csv"
# Print CSV header
echo "\"TIER\",\"START_TIME\",\"END_TIME\",\"ANNOTATION\"" > "$output_csv"
# Get a list of unique tier IDs, sort them and then remove duplicates
tiers=$(xmlstarlet sel -t -v "//TIER/@TIER_ID" "$eaf_file" | sort | uniq)
# For each tier, extract the relevant details
IFS=$'\n'
for tier in $tiers; do
    xmlstarlet sel -t \
        -m "//TIER[@TIER_ID='$tier']/ANNOTATION/ALIGNABLE_ANNOTATION" \
        -v "concat('\"', '$tier', '\"', ',', '\"', /ANNOTATION_DOCUMENT/TIME_ORDER/TIME_SLOT[@TIME_SLOT_ID=current()/@TIME_SLOT_REF1]/@TIME_VALUE, '\"', ',', '\"', /ANNOTATION_DOCUMENT/TIME_ORDER/TIME_SLOT[@TIME_SLOT_ID=current()/@TIME_SLOT_REF2]/@TIME_VALUE, '\"', ',', '\"', ANNOTATION_VALUE, '\"')" \
        -n "$eaf_file" >> "$output_csv"
done
echo "Output saved to $output_csv"

Drugi skrypt wygeneruje próbki i złoży ich nazwy za pomocą danych z wcześniej utworzonego pliku .csv. Ten skrypt zamieni znaki zapytania na “question” i odstępy na podkreślniki (_) by uniknąć problemów przy dalszym przetwarzaniu plików, więc nazwa pliku na podstawie „S-LH-FOCUS,0,1018,in space” to „S-LH-FOCUS_0_1018_in_space__input.mp4”.

In [ ]:
#!/bin/bash

# Check for the correct number of arguments
if [ "$#" -ne 2 ]; then
    echo "Usage: $0 <input_video.mp4> <input.csv>"
    exit 1
fi

input_video="$1"
base_input_video=$(basename "$input_video" .mp4)  # extracting base name without extension
csv_file="$2"

# Check if the files exist
if [[ ! -f "$input_video" || ! -f "$csv_file" ]]; then
    echo "Error: One of the files does not exist."
    exit 2
fi

# Skip the header line in the CSV
tail -n +2 "$csv_file" | while IFS=, read -r part1 start_ms end_ms part4; do
    # Remove any surrounding quotes and potential carriage returns
    part1=$(echo $part1 | tr -d '"\r')
    start_ms=$(echo $start_ms | tr -d '"\r')
    end_ms=$(echo $end_ms | tr -d '"\r')
    part4=$(echo $part4 | tr -d '"\r')

    # Replace spaces with underscores and ? with 'question'
    part1=${part1// /_}
    part4=${part4// /_}
    part4=${part4//\?/question}

    # Construct the filename using original ms format
    filename="${part1}_${start_ms}_${end_ms}_${part4}__${base_input_video}.mp4"

    # Print the ffmpeg command for debugging
    echo "Processing: $filename"
    ffmpeg -i "$input_video" -ss "${start_ms}ms" -t "$(($end_ms - $start_ms))ms" -c:v copy -c:a copy "$filename"
done


Aby uruchomić skrypty, należy zachować je w dwóch osobnych plikach o nazwach eaf2csv.sh i csv2spl.sh, a następnie uruchomić na pliku input.mp4 i input.eaf:
./eaf2csv.sh input.eaf
./csv2spl.sh input.mp4 input.csv

